# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

In [ ]:
# Rohan, Justin, Justin
# Problem Set 3
# QSS 20

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/icefi/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /Users/icefi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.6 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
pharma=str(doj[doj.id=="17-1204"]["contents"])
pharma #raw string

'4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RIC

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
#A

pharma_punct = ''.join(z for z in pharma if z.isalpha() or z.isspace()) #keeping spaces 
pharma_punct = pharma_punct.replace('\xa0', ' ') #replace non breaking spaces with normal spaces
pharma_punct

'    The founder and majority owner of Insys Therapeutics Inc was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain More than  Americans died of synthetic opioid overdoses last year and millions are addicted to opioids And yet some medical professionals would rather take advantage of the addicts than try to help them said Attorney General Jeff Sessions This Justice Department will not tolerate this  We will hold accountable anyone  from street dealers to corporate executives  who illegally contributes to this nationwide epidemic  And under the leadership of President Trump we are fully committed to defeating this threat to the American peopleJohn N Kapoor  of Phoenix Ariz a current member of the Board of Directors of Insys was arrested this morning in Arizona and charged with RICO conspiracy as well as other felonies includi

In [5]:
#B 

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag

tok = word_tokenize(pharma_punct)
tok_part_of_speech = pos_tag(tok)
print(tok_part_of_speech[:10]) #printed the first ten to check 

[nltk_data] Downloading package punkt to /Users/icefi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

[('The', 'DT'), ('founder', 'NN'), ('and', 'CC'), ('majority', 'NN'), ('owner', 'NN'), ('of', 'IN'), ('Insys', 'NNP'), ('Therapeutics', 'NNP'), ('Inc', 'NNP'), ('was', 'VBD')]


In [6]:
#C 

adjectives = [token[0] for token in tok_part_of_speech if token[1] in ["JJS", "JJ", "JJR"]]

countadj = {}
for word in adjectives:
    if word in countadj:
        countadj[word] += 1 
    else:
        countadj[word] = 1

adjsort = sorted(countadj.items(), key=lambda x: x[1], reverse=True)

fivetop = pd.DataFrame(adjsort[:5], columns=["Adjective", "Count"])
fivetop

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,other,3
4,addictive,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
#A: 

pharma = doj.loc[doj.id == "17-1204", "contents"].iloc[0]
pharma_doc = nlp(pharma)

ents = [(entity.text, entity.label_) for entity in pharma_doc.ents]

name_entity_df = pd.DataFrame(ents, columns=["Entity", "Label"]) #put in data frame for now

print(name_entity_df)

                               Entity     Label
0             Insys Therapeutics Inc.       ORG
1                               today      DATE
2                            Fentanyl    PERSON
3                    More than 20,000  CARDINAL
4                           Americans      NORP
..                                ...       ...
141                              U.S.       GPE
142               K. Nathaniel Yeager    PERSON
143  Weinreb’s Health Care Fraud Unit       ORG
144               Susan M. Poswistilo    PERSON
145          Weinreb’s Civil Division       ORG

[146 rows x 2 columns]


In [8]:
# B: 

law_ents = name_entity_df[name_entity_df["Label"] == "LAW"]

unique_laws = law_ents["Entity"].drop_duplicates().reset_index(drop=True) #RICO showed up twice, hence drop duplicates 

unique_laws

0                             RICO
1    the Controlled Substances Act
Name: Entity, dtype: object

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

**#C: 
RICO is a federal law designed to combat organized crime. It can apply to corporate schemes like pharmaceutical kickbacks as well by allowing prosecutors to charge companies / individuals for engagiing in illegal conduct (such as bribery / kickbacks paid to physicians to increase drugs prescribed)**

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
#D 
# We noticed that 20 years and five years are not showing up in pharma_doc.ents even though they are present in pharma_doc (ctrl-f)
# This should be due to spaCy not tagging correctly, hence they don't show up in our output 

matches = []
for ent in pharma_doc.ents:
    if ent.label_ == "DATE" and re.search(r"\byears?\b", ent.text):
        print(f"Entity: {ent.text}, Label: {ent.label_}")
        matches.append(ent.text)

Entity: last year, Label: DATE
Entity: 20 years, Label: DATE
Entity: three years, Label: DATE
Entity: five years, Label: DATE
Entity: three years, Label: DATE


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
#E. 

sentences = list(pharma_doc.sents)

for sent in sentences:
    if re.findall(r"years?\b", sent.text, re.IGNORECASE):
        print("Sentence:", sent.text.strip())

Sentence: "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
Sentence: The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
Sentence: The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.


**If convicted, the CEO could face up to 20 years in prison and three years of supervised release based on the most serious charges outlined in the indictment, if charges for conspiracy, the sentence is no greater than five years and a $250,000 fine**

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
#A 

doj_subset = doj[doj.topics_clean.isin(["Civil Rights", "Project Safe Childhood", "Hate Crimes"])]
len(doj_subset)

717

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
#B: 

objectsentiment = SentimentIntensityAnalyzer()

def sentiment_analysis(pr_string):
    document = nlp(pr_string)
    pr_string_copy = pr_string # copy of pr_string

    for ent in document.ents:
        pattern = re.escape(ent.text)
        pr_string_copy  = re.sub(pattern, "", pr_string_copy, flags=re.IGNORECASE)

    sentiment_scores = objectsentiment.polarity_scores(pr_string_copy)

    return sentiment_scores

In [13]:
doj_subset["sentiment"] = doj_subset["contents"].apply(sentiment_analysis)

In [14]:
doj_subset[["id", "sentiment"]].head()

,id,sentiment
77,17-1235,"{'neg': 0.197, 'neu': 0.755, 'pos': 0.049, 'compound': -0.9931}"
155,15-1522,"{'neg': 0.132, 'neu': 0.8, 'pos': 0.068, 'compound': -0.9325}"
157,16-213,"{'neg': 0.093, 'neu': 0.83, 'pos': 0.077, 'compound': -0.7579}"
162,16-381,"{'neg': 0.123, 'neu': 0.794, 'pos': 0.083, 'compound': -0.9037}"
168,14-464,"{'neg': 0.176, 'neu': 0.781, 'pos': 0.044, 'compound': -0.9864}"


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [15]:
#C

doj_subset_wscore = doj_subset.copy()
sentiment_keys = ["neg","neu","pos","compound"]

In [16]:
for key in sentiment_keys:
    doj_subset_wscore[key] = doj_subset_wscore["sentiment"].apply(lambda s: s[key])

doj_subset_wscore.sort_values("neg", 
                              ascending=False, 
                              inplace=True)


doj_subset_wscore.loc[:, ["id", "contents", "neg"]].head(2)

,id,contents,neg
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.301
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsh

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)

In [17]:
#D 

doj_subset_wscore.groupby("topics_clean").agg({"compound": "mean"})

,compound
topics_clean,
Civil Rights,-0.088592
Hate Crimes,-0.930563
Project Safe Childhood,-0.669454


**#E**

**The variation in compound scores might reflect the emotional tone of each topic, with press releases about hate crimes and child exploitation containing more graphic or distressing content than civil rights cases, leading to more negative overall sentiment.**

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [39]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [40]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

all_stopwords = set(stopwords.words('english')).union(custom_doj_stopwords)

stemmer = SnowballStemmer("english")
def preprocess_text(raw_text):
    
    tokens = word_tokenize(raw_text.lower()) #tokenize and make lowercase 

    filtered = [word for word in tokens
                if word.isalpha() and word not in all_stopwords and len(word) >= 4]
    stemmed = [stemmer.stem(word) for word in filtered]

    return " ".join(stemmed)

In [41]:
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocess_text)

In [42]:
doj_subset_wscore.loc[doj_subset_wscore["id"].isin(["16-718", "16-217"]), ["id", "contents", "processed_text"]]

,id,contents,processed_text
1,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
691,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootin

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [44]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [45]:
#A 

justice_dtm = create_dtm(list_of_strings= doj_subset_wscore.processed_text, #used given function 
                      metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']])

print(justice_dtm.head())

   index       id  compound  topics_clean  aaron  abandon  abbat  abbi  \
0      0   13-312   -0.9983   Hate Crimes      0        0      0     0   
1      1   16-718   -0.9964  Civil Rights      0        0      0     0   
2      2   11-626   -0.9986   Hate Crimes      0        0      0     0   
3      3   14-248   -0.9935   Hate Crimes      0        0      0     0   
4      4  10-1194   -0.9990   Hate Crimes      0        0      0     0   

   abbott  abdomen  ...  zamora  zane  zealand  zealous  zeeman  zero  zobel  \
0       0        0  ...       0     0        0        0       0     0      0   
1       0        0  ...       0     0        0        0       0     0      0   
2       0        0  ...       0     0        0        0       0     0      0   
3       0        0  ...       0     0        0        0       0     0      0   
4       0        0  ...       0     0        0        0       0     0      0   

   zone  zunggeemog  zwengel  
0     0           0        0  
1     0     

In [46]:
#Create topwords function 

def get_topwords(subset):
    topwords = subset.drop(columns=['topics_clean', 'index', 'id', 'compound']).sum()
    topwords = topwords.sort_values(ascending = False).head(10) # because we get top 10 terms
    return topwords

In [47]:
#B

cutoff_val = justice_dtm.iloc[:, 2].quantile(q=0.95)

top_five_subset = justice_dtm.loc[justice_dtm.iloc[:, 2] >= cutoff_val]

get_topwords(top_five_subset)

agreement     182
enforc        137
state         115
ensur         108
communiti     100
disabl         97
polic          89
general        87
student        86
settlement     83
dtype: int64

In [48]:
#C 
low_cutoff = justice_dtm.iloc[:, 2].quantile(0.05)

bottom_five_subset = justice_dtm.loc[justice_dtm.iloc[:, 2] <= low_cutoff]

get_topwords(bottom_five_subset)

assault     200
victim      166
crime       166
hate        128
defend      127
offic       119
sentenc     104
charg       100
anderson     93
guilti       92
dtype: int64

In [49]:
#D 

for topic_label in justice_dtm["topics_clean"].unique():
    print(f"\nTop words for topic: {topic_label}")
    
    topic_data = justice_dtm[justice_dtm["topics_clean"] == topic_label]
    
    get_topwords(topic_data)


Top words for topic: Hate Crimes


victim      590
crime       533
prosecut    476
hate        472
defend      459
sentenc     455
charg       452
guilti      430
feder       426
said        424
dtype: int64


Top words for topic: Civil Rights


offic        627
hous         620
discrimin    541
enforc       531
disabl       509
said         497
feder        475
violat       470
state        443
general      408
dtype: int64


Top words for topic: Project Safe Childhood


child          1018
exploit         698
sexual          570
safe            476
childhood       472
project         472
pornographi     447
children        416
crimin          404
prosecut        374
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [50]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize

tokenized_texts = [word_tokenize(text) for text in doj_subset_wscore["processed_text"]]

dictionary = corpora.Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    passes=10,
    random_state=42
)

for idx, topic in lda_model.show_topics(num_topics=3, num_words=15, formatted=True):
    print(f"\n: Topic {idx + 1}:\n{topic}")


: Topic 1:
0.011*"victim" + 0.011*"charg" + 0.010*"prosecut" + 0.009*"sentenc" + 0.009*"defend" + 0.009*"crime" + 0.009*"guilti" + 0.009*"feder" + 0.008*"indict" + 0.008*"said" + 0.008*"hate" + 0.007*"assault" + 0.007*"prison" + 0.007*"year" + 0.007*"investig"

: Topic 2:
0.022*"child" + 0.015*"exploit" + 0.014*"sexual" + 0.011*"safe" + 0.010*"project" + 0.010*"childhood" + 0.010*"crimin" + 0.010*"sentenc" + 0.010*"prosecut" + 0.010*"offic" + 0.010*"pornographi" + 0.009*"victim" + 0.009*"children" + 0.007*"investig" + 0.007*"abus"

: Topic 3:
0.011*"hous" + 0.010*"discrimin" + 0.009*"disabl" + 0.007*"enforc" + 0.007*"said" + 0.007*"agreement" + 0.007*"state" + 0.006*"court" + 0.006*"alleg" + 0.006*"violat" + 0.005*"feder" + 0.005*"general" + 0.005*"requir" + 0.005*"settlement" + 0.005*"individu"


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [51]:
#A 

doc_topic_probs = [lda_model.get_document_topics(bow, minimum_probability=0) for bow in corpus]

assert len(doc_topic_probs) == len(doj_subset_wscore)

In [52]:
#B 

topic_distributions = [
    [prob for _, prob in doc] for doc in doc_topic_probs
]

topic_df = pd.DataFrame(topic_distributions, columns=["topic_1", "topic_2", "topic_3"])

doj_subset_wscore = pd.concat([doj_subset_wscore.reset_index(drop=True), topic_df], axis=1)

doj_subset_wscore["top_topic"] = topic_df.idxmax(axis=1)

doj_subset_wscore["top_topic"] = doj_subset_wscore["top_topic"].str.extract("(\d)").astype(int)

In [53]:
#C 

# For reference, topic 1 is hate crimes, topic 2 is project safe childhood, and topic 3 is civil rights

topic_breakdown = pd.crosstab(
    doj_subset_wscore["topics_clean"],
    doj_subset_wscore["top_topic"],
    normalize="index")

print(topic_breakdown)

top_topic                      1         2         3
topics_clean                                        
Civil Rights            0.304918  0.026230  0.668852
Hate Crimes             0.991870  0.000000  0.008130
Project Safe Childhood  0.006024  0.993976  0.000000


**Based on the topic distribution, Project Safe Childhood maps most cleanly onto a single estimated topic, with 96.99% of documents classified under Topic 2—likely due to its consistent language around child exploitation and safety. In contrast, Hate Crimes is more split, with 50.81% in Topic 1 and 33.74% in Topic 2, suggesting overlapping vocabulary with civil rights or legal terminology. Similarly, Civil Rights press releases are more dispersed, with 67.87% in Topic 3 but also 18.03% and 14.10% in Topics 1 and 2 respectively, indicating a broader range that spans justice, enforcement, and discrimination**

In [54]:
# Press release example 1 (Civil Rights) 

doj_subset_wscore[doj_subset_wscore.id == "16-1379"]["contents"]

# this civil rights case uses words like race, color, religion, and discrimination which can fit into both hate crimes and civil rights. hence there is some overlap between these categories. 

553    The Justice Department filed a civil lawsuit today against the village of Tinley Park, Illinois, alleging that the suburb of Chicago violated the Fair Housing Act when it refused to approve a low-income housing development in response to race-based community opposition. The suit alleges that in 2015, the Buckeye Community Hope Foundation sought to build a 47-unit apartment complex for low-income individuals just outside of Tinley Park’s downtown.  The project would be financed through Low Income Housing Tax Credits administered by the Illinois Housing and Development Authority.  The suit alleges that Tinley Park discriminated against prospective tenants of the proposed development when it refused to approve the project despite the Tinley Park Planning Department’s finding that the project was in “precise conformance” with the applicable building requirements.  Under Tinley Park’s zoning ordinances, Tinley Park’s Plan Commission should have approved the project and allowed constr

In [55]:
# Press release example 2 (Project Safe Childhood)

doj_subset_wscore[doj_subset_wscore.id == "12-598"]["contents"]

# Words such as child porography and child sexual abuse make it so that it is easy to
# classify this into the project safe childhood "bucket"

64        WASHINGTON – A Manalapan, N.J., woman pleaded guilty today to producing child pornography by sexually abusing a five-year-old girl on more than one occasion and streaming footage of a sexual assault over the Internet, Assistant Attorney General Lanny A. Breuer of the Justice Department’s Criminal Division and New Jersey U.S. Attorney Paul J. Fishman announced.           Jennifer Mahoney, 33, pleaded guilty to one count of sexual exploitation of a child.  She entered her guilty plea in Trenton, N.J., federal court before U.S. District Judge Mary L. Cooper.           “Ms. Mahoney sexually abused a five-year-old girl and streamed footage of this abuse to others over the Internet,” said Assistant Attorney General Breuer.  “Now that she has pleaded guilty to these reprehensible crimes, she faces a minimum of 15 years in prison.  No prison sentence can repair the damage she has caused, or restore the innocence of the child she abused.  But, she and other child predators should know

In [56]:
# Press release example 3 (hate crime) 

doj_subset_wscore[doj_subset_wscore.id == "11-626"]["contents"]

# even though this is a hate crime, there is language like civil rights, victim, and convicted, which could muddle the classification 

2    WASHINGTON – The Justice Department announced today that Sean Popejoy, 19, of Green Forest, Ark., pleaded guilty in federal court to one count of committing a federal hate crime and one count of conspiring to commit a federal hate crime. This is the first conviction for a violation of the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, which was enacted in October 2009.   Information presented during the plea hearing established that in the early morning hours of June 20, 2010, Popejoy admitted that he was part of a conspiracy to threaten and injure five Hispanic men who had pulled into a gas station parking lot.  The co-conspirators pursued the victims in a truck. When the co-conspirators caught up to the victims, Popejoy leaned outside of the front passenger window and waived a tire wrench at the victims and continued to threaten and hurl racial epithets at the victims. The co-conspirator rammed into the victims' car, which caused the victims’ car to cross the opp

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [57]:
#A 

def create_bigram_onedoc(processed_text):
    tokens = processed_text.split()
    bigrams = [f"{w1}_{w2}" for w1, w2 in zip(tokens, tokens[1:])]
    return " ".join(bigrams)

doj_subset_wscore["processed_text_bigrams"] = doj_subset_wscore["processed_text"].apply(create_bigram_onedoc)

In [58]:
#B

doj_subset_wscore.loc[doj_subset_wscore["id"] == "16-217", ["id", "processed_text", "processed_text_bigrams"]]

,id,processed_text,processed_text_bigrams
691,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_applaud applaud_citi citi_commiss commiss_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_find find

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [59]:
#C 

dtm_bigram = create_dtm(
    list_of_strings=doj_subset_wscore["processed_text_bigrams"],
    metadata=doj_subset_wscore[["topics_clean", "compound", "id"]]
)

dtm_bigram.head()
dtm_bigram.shape

,index,topics_clean,compound,id,aaron_ford,aaron_latham,aaron_mcgrath,aaron_parrish,aaron_polster,aaron_rice,...,zone_request,zone_restrict,zone_student,zone_time,zone_today,zone_varianc,zunggeemog_noel,zunggeemog_prompt,zunggeemog_write,zwengel_princeton
0,0,Hate Crimes,-0.9983,13-312,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Civil Rights,-0.9964,16-718,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Hate Crimes,-0.9986,11-626,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Hate Crimes,-0.9935,14-248,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Hate Crimes,-0.9990,10-1194,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(717, 71336)

In [60]:
#D: 

print("Unigram DTM shape:", justice_dtm.shape)
print("Bigram DTM shape:", dtm_bigram.shape)

Unigram DTM shape: (717, 6760)
Bigram DTM shape: (717, 71336)


E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

**Explanation:**
    
**The bigram matrix has more dimensions (i.e., more columns) than the unigram matrix because it includes all unique, consecutive pairs of words in the dataset,
which increases the number of values stored. While the unigram only contains individual words,
the bigram DTM contains combinations/the number of combinations is much higher.**

In [61]:
#E 

for topic_label in dtm_bigram["topics_clean"].unique():
    print(f"\nTop words for topic: {topic_label}")
    
    topic_data = dtm_bigram[dtm_bigram["topics_clean"] == topic_label]
    
    get_topwords(topic_data)


plead_guilti         551
safe_childhood       472
project_safe         472
child_pornographi    446
hate_crime           378
year_prison          315
unit_state           289
child_exploit        280
fair_hous            253
special_agent        241
dtype: int64

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 